In [ ]:
# Notebook B: Classification avec Deep Learning
# Classification des chiffres manuscrits avec un réseau de neurones

# Partie 1: Importation des bibliothèques
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from google.colab import output
output.enable_custom_widget_manager()
import ipywidgets as widgets

# Vérifier la version de TensorFlow
print(f"TensorFlow version: {tf.__version__}")

# Partie 2: Chargement et exploration des données
print("Chargement du jeu de données MNIST...")
# Utilisation du jeu de données MNIST intégré à TensorFlow
(X_train_full, y_train_full), (X_test_full, y_test_full) = tf.keras.datasets.mnist.load_data()

# Normalisation des valeurs de pixels entre 0 et 1
X_train_full = X_train_full / 255.0
X_test_full = X_test_full / 255.0

# Exploration des données
print(f"Dimensions des données d'entraînement: {X_train_full.shape}")
print(f"Dimensions des données de test: {X_test_full.shape}")
print(f"Nombre de classes: {len(np.unique(y_train_full))}")

# Affichage de quelques exemples
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X_train_full[i], cmap='gray')
    plt.title(f"Label: {y_train_full[i]}")
    plt.axis('off')
plt.tight_layout()
plt.suptitle("Exemples de chiffres manuscrits", y=1.05)
plt.show()

# Partie 3: Préparation des données pour Deep Learning

print("\n--- Préparation des données pour le réseau de neurones ---")
print("Pour le Deep Learning, nous n'avons pas besoin d'extraire manuellement des caractéristiques.")

# Utiliser un sous-ensemble des données pour accélérer la démonstration (même taille que le Random Forest)
n_samples = 10000
X_train = X_train_full[:n_samples]
y_train = y_train_full[:n_samples]
X_test = X_test_full[:2000]
y_test = y_test_full[:2000]

# Pour le Deep Learning, il nous faut juste redimensionner les images
print("Redimensionnement des images...")
print(f"Forme originale: {X_train.shape}")

# Aplatir les images 28x28 en vecteurs de 784 pixels
X_train_flat = X_train.reshape(X_train.shape[0], 28*28)
X_test_flat = X_test.reshape(X_test.shape[0], 28*28)

print(f"Forme après redimensionnement: {X_train_flat.shape}")

# One-hot encoding des labels
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

print("Encodage one-hot des labels")
print(f"Forme originale des labels: {y_train.shape}")
print(f"Forme après encodage one-hot: {y_train_cat.shape}")

# Partie 4: Création du modèle de Deep Learning

print("\n--- Création du modèle de réseau de neurones ---")

# Paramètres du modèle - vous pouvez les modifier
n_hidden1 = 128  # Nombre de neurones dans la première couche cachée
n_hidden2 = 64   # Nombre de neurones dans la seconde couche cachée
learning_rate = 0.001  # Taux d'apprentissage
dropout_rate = 0.2  # Taux de dropout pour la régularisation
n_epochs = 10  # Nombre d'époques d'entraînement
batch_size = 32  # Taille du batch

# Création du modèle
model = Sequential([
    # Couche d'entrée: 784 neurones (un par pixel)
    # Première couche cachée
    Dense(n_hidden1, activation='relu', input_shape=(784,)),
    Dropout(dropout_rate),
    # Deuxième couche cachée
    Dense(n_hidden2, activation='relu'),
    Dropout(dropout_rate),
    # Couche de sortie: 10 neurones (un par classe)
    Dense(10, activation='softmax')
])

# Compilation du modèle
optimizer = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Affichage du résumé du modèle
model.summary()

# Partie 5: Entraînement du modèle

print("\n--- Entraînement du modèle de Deep Learning ---")

# Mesure du temps d'entraînement
start_time = time.time()
print("Entraînement du modèle en cours...")

# Entraînement du modèle
history = model.fit(
    X_train_flat, y_train_cat,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_split=0.2,
    verbose=1
)

end_time = time.time()
training_time = end_time - start_time
print(f"Temps d'entraînement: {training_time:.2f} secondes")

# Partie 6: Visualisation de l'apprentissage

print("\n--- Visualisation de l'apprentissage ---")

# Tracer l'évolution de la précision et de la perte
plt.figure(figsize=(12, 4))

# Évolution de la précision
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Entraînement')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Évolution de la précision')
plt.xlabel('Époque')
plt.ylabel('Précision')
plt.legend()

# Évolution de la perte
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Entraînement')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Évolution de la perte')
plt.xlabel('Époque')
plt.ylabel('Perte')
plt.legend()

plt.tight_layout()
plt.show()

# Partie 7: Évaluation du modèle

print("\n--- Évaluation du modèle de Deep Learning ---")

# Évaluation sur l'ensemble de test
test_loss, test_accuracy = model.evaluate(X_test_flat, y_test_cat, verbose=0)
print(f"Précision sur l'ensemble de test: {test_accuracy*100:.2f}%")

# Prédictions sur l'ensemble de test
y_pred_prob = model.predict(X_test_flat)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calcul des métriques
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("\nMatrice de confusion:")
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Prédictions')
plt.ylabel('Valeurs réelles')
plt.title('Matrice de confusion')
plt.show()

print("\nRapport de classification:")
print(class_report)

# Partie 8: Visualisation des erreurs

print("\n--- Analyse des erreurs ---")

# Identifier les erreurs
error_indices = np.where(y_pred != y_test)[0]
n_errors = min(10, len(error_indices))  # Afficher max 10 erreurs

if n_errors > 0:
    plt.figure(figsize=(12, 4))
    for i, idx in enumerate(error_indices[:n_errors]):
        plt.subplot(2, 5, i + 1)
        plt.imshow(X_test[idx], cmap='gray')
        plt.title(f"Réel: {y_test[idx]}\nPrédit: {y_pred[idx]}")
        plt.axis('off')
    plt.tight_layout()
    plt.suptitle("Exemples d'erreurs de classification", y=1.05)
    plt.show()
else:
    print("Aucune erreur trouvée dans l'échantillon de test!")

# Partie 9: Visualisation des poids

print("\n--- Visualisation des poids appris ---")

# Récupérer les poids de la première couche
weights = model.layers[0].get_weights()[0]  # [784, n_hidden1]

# Visualiser quelques filtres (neurones de la première couche)
plt.figure(figsize=(12, 6))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    # Redimensionner les poids du neurone en 28x28
    neuron_weights = weights[:, i].reshape(28, 28)
    plt.imshow(neuron_weights, cmap='coolwarm')
    plt.title(f"Neurone {i+1}")
    plt.axis('off')
plt.tight_layout()
plt.suptitle("Poids appris par les neurones de la première couche", y=1.05)
plt.show()

print("Ces visualisations montrent ce que 'recherchent' les neurones de la première couche dans les images.")

# Partie 10: Défi de généralisation

print("\n--- Défi de généralisation ---")
print("Nous allons maintenant tester le modèle sur des chiffres légèrement modifiés pour évaluer sa capacité de généralisation.")

# Fonction pour ajouter du bruit aux images
def add_noise(images, noise_level=0.2):
    noisy_images = images.copy()
    noise = np.random.normal(0, noise_level, images.shape)
    noisy_images = noisy_images + noise
    # Assurer que les valeurs restent entre 0 et 1
    noisy_images = np.clip(noisy_images, 0, 1)
    return noisy_images

# Fonction pour appliquer une rotation aux images
def rotate_images(images, max_angle=15):
    from scipy.ndimage import rotate
    rotated_images = np.zeros_like(images)
    for i, img in enumerate(images):
        angle = np.random.uniform(-max_angle, max_angle)
        rotated = rotate(img, angle, reshape=False)
        rotated_images[i] = rotated
    return rotated_images

# Créer un jeu de données modifié
print("Création d'un jeu de données avec des chiffres modifiés...")

# Utiliser un nouvel ensemble de données pour ce test
X_new = X_test_full[2000:4000]
y_new = y_test_full[2000:4000]

# Appliquer des transformations
X_new_noisy = add_noise(X_new, noise_level=0.2)
X_new_rotated = rotate_images(X_new, max_angle=15)

# Visualiser quelques exemples
plt.figure(figsize=(12, 8))
for i in range(5):
    # Original
    plt.subplot(3, 5, i + 1)
    plt.imshow(X_new[i], cmap='gray')
    plt.title(f"Original: {y_new[i]}")
    plt.axis('off')
    
    # Avec bruit
    plt.subplot(3, 5, i + 6)
    plt.imshow(X_new_noisy[i], cmap='gray')
    plt.title("Avec bruit")
    plt.axis('off')
    
    # Avec rotation
    plt.subplot(3, 5, i + 11)
    plt.imshow(X_new_rotated[i], cmap='gray')
    plt.title("Avec rotation")
    plt.axis('off')

plt.tight_layout()
plt.suptitle("Exemples de chiffres modifiés", y=1.02)
plt.show()

# Préparation des données modifiées
X_new_flat = X_new.reshape(X_new.shape[0], 28*28)
X_new_noisy_flat = X_new_noisy.reshape(X_new_noisy.shape[0], 28*28)
X_new_rotated_flat = X_new_rotated.reshape(X_new_rotated.shape[0], 28*28)

# Évaluer le modèle sur les données modifiées
print("\nÉvaluation sur les données originales:")
y_new_pred = np.argmax(model.predict(X_new_flat), axis=1)
accuracy_original = accuracy_score(y_new, y_new_pred)
print(f"Précision sur données originales: {accuracy_original*100:.2f}%")

print("\nÉvaluation sur les données avec bruit:")
y_new_noisy_pred = np.argmax(model.predict(X_new_noisy_flat), axis=1)
accuracy_noisy = accuracy_score(y_new, y_new_noisy_pred)
print(f"Précision sur données bruitées: {accuracy_noisy*100:.2f}%")

print("\nÉvaluation sur les données avec rotation:")
y_new_rotated_pred = np.argmax(model.predict(X_new_rotated_flat), axis=1)
accuracy_rotated = accuracy_score(y_new, y_new_rotated_pred)
print(f"Précision sur données pivotées: {accuracy_rotated*100:.2f}%")

print("\nComparaison des précisions:")
print(f"Précision sur données originales: {accuracy_original*100:.2f}%")
print(f"Précision sur données bruitées: {accuracy_noisy*100:.2f}%")
print(f"Précision sur données pivotées: {accuracy_rotated*100:.2f}%")

# Partie 11: Conclusions et réflexion

print("\n--- Conclusions sur le Deep Learning ---")
print("""
Points forts du réseau de neurones:
- Apprentissage automatique des caractéristiques (pas de feature engineering manuel)
- Bonnes performances sur les données originales et transformées
- Capacité à capturer des motifs complexes

Limites:
- Temps d'entraînement généralement plus long que les méthodes classiques
- Plus de paramètres à régler
- Risque de surapprentissage sur petits ensembles de données
- Interprétabilité plus difficile

Questions pour la réflexion:
1. Pourquoi le Deep Learning gère-t-il mieux les transformations des données?
2. Comment pourrions-nous améliorer encore les performances du modèle?
3. Quels types de problèmes sont particulièrement adaptés au Deep Learning?
""")

# Partie 12: Widget interactif pour tester le modèle

print("\n--- Testez le modèle vous-même ---")

def test_dl_model(digit_idx):
    if digit_idx < len(X_test):
        # Afficher l'image
        img = X_test[digit_idx]
        plt.figure(figsize=(6, 6))
        plt.imshow(img, cmap='gray')
        plt.title(f"Chiffre à classifier")
        plt.axis('off')
        plt.show()
        
        # Faire la prédiction
        img_flat = img.reshape(1, 784)
        prediction_prob = model.predict(img_flat)[0]
        prediction = np.argmax(prediction_prob)
        real_label = y_test[digit_idx]
        
        # Afficher les probabilités
        plt.figure(figsize=(10, 4))
        plt.bar(range(10), prediction_prob)
        plt.xticks(range(10))
        plt.xlabel('Chiffre')
        plt.ylabel('Probabilité')
        plt.title('Probabilités prédites pour chaque chiffre')
        plt.show()
        
        print(f"Prédiction du modèle Deep Learning: {prediction}")
        print(f"Étiquette réelle: {real_label}")
        print(f"Prédiction {'correcte' if prediction == real_label else 'incorrecte'}")
        
        # Afficher les 3 prédictions les plus probables
        top3_idx = np.argsort(prediction_prob)[-3:][::-1]
        print("\nTop 3 des prédictions:")
        for i, idx in enumerate(top3_idx):
            print(f"{i+1}. Chiffre {idx}: {prediction_prob[idx]*100:.2f}%")
    else:
        print("Index hors limites!")

# Créer un slider pour sélectionner un chiffre à tester
digit_selector = widgets.IntSlider(
    value=0,
    min=0,
    max=len(X_test)-1,
    step=1,
    description='Index:',
    continuous_update=False
)

# Bouton pour exécuter le test
test_button = widgets.Button(description="Tester")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        test_dl_model(digit_selector.value)

test_button.on_click(on_button_clicked)

# Afficher les widgets
display(widgets.HBox([digit_selector, test_button]))
display(output)

print("Utilisez le slider pour sélectionner un index et cliquez sur 'Tester' pour classifier le chiffre correspondant.")

# Partie 13: Comparaison avec le modèle Random Forest

print("\n--- Comment comparer avec le modèle de Machine Learning classique? ---")
print("""
Après avoir exploré ce modèle de Deep Learning, comparez-le avec le modèle Random Forest (Notebook A) sur les aspects suivants:

1. Préparation des données:
   - Random Forest: nécessite une réduction de dimension (PCA)
   - Deep Learning: travaille directement avec les pixels bruts

2. Architecture et hyperparamètres:
   - Random Forest: nombre d'arbres, profondeur, critères de division
   - Deep Learning: nombre de couches, neurones, fonctions d'activation, dropout

3. Temps d'entraînement:
   - Lequel est plus rapide sur ce jeu de données?
   - Comment cela évoluerait-il avec plus de données?

4. Performance:
   - Sur les données originales
   - Sur les données modifiées (bruit, rotation)

5. Interprétabilité:
   - Facilité à comprendre ce que le modèle a appris
   - Visualisation des caractéristiques importantes

Remplissez le tableau comparatif fourni pour structurer votre analyse.
""")